In [6]:
from google.cloud import bigquery

In [7]:
import os, io, math, itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import hot
import seaborn as sns
from datetime import datetime
from math import sqrt
from scipy import stats

In [8]:
def get_raw(client, sql):
    """
    get Chicago taxi fare raw data from BigQuery public dataset
    """
    raw = client.query(sql).to_dataframe()
    return raw

In [9]:
def data_types(df):
    '''
    Show data types of each column
    '''
    print (df.dtypes)


In [10]:
def data_describe(df):
    '''
    Show the descriptive statistics (e.g., mean, min, max, count) of each column
    '''
    print (df.describe())    

In [11]:

def missing_values(df):
    '''
    Show the number of missing values for each column
    '''
    print(df.isnull().sum())

In [12]:
def share_rides(df):
    '''
    Check how many rides are share rides
    If two unique_key have the same taxi_id and trip_start_timestamp, it is considered a shared ride
    '''
    aggregations = {
        'unique_key':{
            'passengers': 'count'
        },
        'fare':{
            'max_fare': 'max', 
            'min_fare': 'min'
        }
    }
    
    df_2=df
    df_share_rides = df_2[['unique_key', 'taxi_id', 'trip_start_timestamp', 'fare']].groupby(['taxi_id', 'trip_start_timestamp']).agg(aggregations)
    df_share_rides.columns = df_share_rides.columns.get_level_values(1)
    
    # For modeling purposes, only include the ones that are share rides, to get an accurate estimation of the fare
    df_3 = df_2.merge(df_share_rides, left_on=['taxi_id', 'trip_start_timestamp'], right_on=['taxi_id', 'trip_start_timestamp'], how='left')
    df_4 = df_3.loc[df_3.passengers > 1]
    return (df_4)


In [14]:
# set up a client 
client = bigquery.Client()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started